In [ ]:
!pip install segmentation-models albumentations

In [ ]:
%load_ext autoreload
%autoreload 2

from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, Input, MaxPooling2D, concatenate, AveragePooling1D, Reshape, Activation, add, Conv2DTranspose, BatchNormalization, UpSampling2D, SeparableConv2D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import CategoricalCrossentropy, Reduction, BinaryCrossentropy
from tensorflow.keras.layers import Lambda
from tensorflow.keras.activations import softmax
from tensorflow import roll, norm, add
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.applications as A
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import numpy as np
import os
import fnmatch
import cv2
import json

import include.supervisely_parser as svp
import include.grid_parser as gp

import segmentation_models as sm # for simple segmentation architecture
import albumentations as Alb

In [ ]:
annotation_path = './annotation_v0.2/'
image_path = './data/Mikrowunderland_1k/'
model_path = './model/ld_autosys.h5'
number_classes = 5 # outer_l, outer_t, outer_r, middle_curb
x_values = 192
y_values = 48

input_width = 434
input_height = 150


In [ ]:
def preprocess(img):
    height, _, _ = img.shape
    img = img[int(height/2):,:,:]
    return img
def preprocess_gray(img):
    height, _ = img.shape
    img = img[int(height/2):,:]
    return img
transform = Alb.Compose([
    Alb.ShiftScaleRotate(p=1.0),
    Alb.RandomContrast(),
    Alb.RandomBrightness()
])

In [ ]:
images = []
segmented_debug = []
data = []

file_list = os.listdir(annotation_path)
pattern = '*.json'
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        ann_data_path = os.path.join(annotation_path, filename)
        image_name = os.path.splitext(filename)[0]
        image_data_path = os.path.join(image_path, image_name)
        
        # extract lanes from ann data
        lanes = svp.getPoints(ann_data_path)
        if len(lanes.keys()) > 0:
            # only if this image has annotation data
            img = cv2.imread(image_data_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            segmented_data = svp.drawLanes((img.shape[0], img.shape[1]), lanes)
            segmented_debug_image = svp.drawDebugImage(np.zeros(img.shape), lanes)
            merged = cv2.merge([cv2.resize(preprocess_gray(segmented), (x_values, y_values)) for segmented in segmented_data])
            #grid = gp.segmented_image_into_grid_space(merged, grid_size=(y_values, x_values), window_size_x=2)
            #grid = np.reshape(grid, (-1, y_values, x_values, number_classes))
            preprocessed_image = img
            img = preprocess(img)
            img = cv2.resize(img, (input_width, input_height))
            segmented_debug_image = preprocess(segmented_debug_image)
            images.append(img)
            data.append(merged)
            segmented_debug.append(segmented_debug_image)


            # augment this dataset
            transformed = transform(image=preprocessed_image, masks=segmented_data)
            new_image = transformed['image']
            new_image = preprocess(new_image)
            new_image = cv2.resize(new_image, (input_width, input_height))
            images.append(cv2.resize(new_image, (input_width, input_height)))
            new_masks = np.array(transformed['masks'])
            new_masks = cv2.merge([cv2.resize(preprocess_gray(mask), (x_values, y_values)) for mask in new_masks])
            #new_masks = cv2.merge([mask for mask in segmented_data])
            data.append(new_masks)
            segmented_debug.append(segmented_debug_image)
            
            


In [ ]:
c = 10
f, axs = plt.subplots(c, 7, figsize=(50,c*3))
i = 0
print(len(images))
for y in range(c):
    axs[y,0].imshow(images[i])
    axs[y,1].imshow(segmented_debug[i])
    axs[y,2].imshow(data[i][:,:,0], cmap='plasma')
    axs[y,3].imshow(data[i][:,:,1], cmap='plasma')
    axs[y,4].imshow(data[i][:,:,2], cmap='plasma')
    axs[y,5].imshow(data[i][:,:,3], cmap='plasma')
    axs[y,6].imshow(data[i][:,:,4], cmap='plasma')
    i += 1

In [ ]:
images = np.array(images)
data = np.array(data)

img_train, img_valid, data_train, data_valid = train_test_split(images, data, test_size=0.2)
print("Training data: %d %d\nValidation data: %d %d" % (len(img_train), len(data_train), len(img_valid), len(data_valid)))
print(img_train.shape)
print(data_train.shape)

In [ ]:
# run only if segmentation models will be used
BACKBONE = 'inceptionv3'
K.set_image_data_format('channels_last')
preprocess_input = sm.get_preprocessing(BACKBONE)

img_train = preprocess_input(img_train)
img_valid = preprocess_input(img_valid)

print(img_train.shape)
print(data_train.shape)

In [ ]:
def test_custom(name):
   # base_model = A.ResNet50V2(include_top=False, weights="imagenet", input_shape=(input_height,input_width,3)) 
    base_model = A.InceptionV3(include_top=False, weights="imagenet", input_shape=(input_height,input_width,3)) 
    #base_model = A.VGG16(include_top=False, weights="imagenet", input_shape=(input_height,input_width,3))
    pool = Conv2D(512, (1, 1))(base_model.output)
    x = Conv2DTranspose(256, 3, strides=(2,2), padding='same')(pool)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    for filters in [128, 64, 32]:
        x = Conv2DTranspose(filters, 3, strides=(2,2), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        x = Dropout(0.4)(x)
    output = Conv2D(number_classes, 1)(x)
    output = Activation('softmax')(output)
    
    model = Model(inputs=base_model.input, outputs=output, name=name)
    optimizer = Adam(lr=1e-4) # lr is learning rate
    loss = sm.losses.CategoricalCELoss() + sm.losses.DiceLoss()
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[sm.metrics.iou_score]) # mean squared error because it is a regression problem
    #plot_model(model, to_file='%s.png' % (name))
    return model


def custom_unet():
    inputs = Input(shape=(input_height,input_width,3))
    ### [First half of the network: downsampling inputs] ###
    # Entry block
    x = Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    previous_block_activation = x  # Set aside residual
    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(filters, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = tf.keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("relu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        # Project residual
        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding="same")(residual)
        x = tf.keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = Conv2D(number_classes, 3, activation="softmax", padding="same")(x)
    # Define the model
    model = Model(inputs, outputs)
    loss = sm.losses.CategoricalCELoss()
    loss = sm.losses.DiceLoss()
    model.compile(optimizer="rmsprop", loss=loss, metrics=[sm.metrics.iou_score])
    return model
def segmentation_model():
    model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=number_classes, activation='softmax')
    optimizer = Adam(lr=4e-4) # lr is learning rate
    loss = sm.losses.DiceLoss()
    loss = sm.losses.CategoricalCELoss()
    model.compile(optimizer = optimizer, loss=loss, metrics=[sm.metrics.iou_score])
    return model
    
model = test_custom('AUTOSYS_LANEDETECTION_CLS')
#model = custom_unet()
#model = segmentation_model()

print(model.summary())

In [ ]:
# Hyperparamter
my_batch_size = 8
my_epochs = 60
# checkpoint
filepath="weights-improvement-{epoch:02d}-{val_loss:.2f}.h5"
#checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#callbacks_list = [checkpoint]

history = model.fit(img_train, data_train,
                   batch_size=my_batch_size,
                   epochs=my_epochs,
                   verbose=1,
                   validation_data=(img_valid, data_valid))#,
                   #callbacks=callbacks_list)
model.save(model_path)

In [ ]:
# Plot training & validation iou_score values
plt.figure(figsize=(30, 5))
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.savefig('inceptionv3_60epoch_4conv2d.png')
plt.show()



In [ ]:
# load model only if not trained in this session\
model = load_model('./model/ld_autosys2.h5', custom_objects={'loss_total':loss_total})

In [ ]:
# load some test images
file_list = os.listdir(annotation_path)
pattern = '*.json'
cnt = 15
test_imgs = []

for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        ann_data_path = os.path.join(annotation_path, filename)
        image_name = os.path.splitext(filename)[0]
        image_data_path = os.path.join(image_path, image_name)
        # extract lanes from ann data
        lanes = svp.getPoints(ann_data_path)
        # only images without annotation
        if len(lanes.keys()) == 0:
            img = cv2.imread(image_data_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = preprocess(img)
            img = cv2.resize(img, (input_width, input_height))

            test_imgs.append(img)
            cnt -= 1
            if cnt == 0:
                break

In [ ]:
predictions = model.predict(np.array(test_imgs))

thres_value = 0.3

def postprocess_channel(img):
    #img = cv2.medianBlur(img, 3)
    _, img = cv2.threshold(img,thres_value,1.0,cv2.THRESH_BINARY)
    return img

f, axs = plt.subplots(len(predictions), 6, figsize=(50,len(predictions)*3))
for i, prediciton in enumerate(predictions):
    #test_img = cv2.resize(test_imgs[i], (x_values, y_values))
    test_img = test_img/255
    test_img = test_img.astype(np.float32)
    predicted_lanes = np.sum([postprocess_channel(prediciton[:,:,i]) for i in range(4)], axis=0)
    predicted_lanes = cv2.merge([np.zeros_like(predicted_lanes), np.zeros_like(predicted_lanes), predicted_lanes])
    predicted_lanes = cv2.resize(predicted_lanes, (test_img.shape[1], test_img.shape[0]))
    overlay_image = cv2.addWeighted(test_img, 0.5, predicted_lanes, 0.5, 0)
    axs[i,0].imshow(overlay_image)
    #axs[i,0].imshow(test_imgs[i])
    axs[i,1].imshow(postprocess_channel(prediciton[:,:,0]), cmap='plasma')
    axs[i,2].imshow(postprocess_channel(prediciton[:,:,1]), cmap='plasma')
    axs[i,3].imshow(postprocess_channel(prediciton[:,:,2]), cmap='plasma')
    axs[i,4].imshow(postprocess_channel(prediciton[:,:,3]), cmap='plasma')
    axs[i,5].imshow(postprocess_channel(prediciton[:,:,4]), cmap='plasma')